In [ ]:
import kaggle as kg
import os
import pandas as pd
import praw
import requests
import csv
import snscrape.modules.twitter as sntwitter
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
# do pip install pytrends
from pytrends.request import TrendReq

## Environment setup
    #Kaggle
os.environ['KAGGLE_USERNAME'] = '{KAGGLE_USERNAME}'
os.environ['KAGGLE_KEY'] = '{KAGGLE_KEY}'
kg.api.authenticate()
    # VADER
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
    # Reddit
reddit = praw.Reddit(
    client_id="YOUR_CLIENT_ID",
    client_secret="YOUR_CLIENT_SECRET",
    user_agents="YOUR_USER_AGENT"
)

## dataset1 Historical-min/min-BTC
data_1 = "mczielinski/bitcoin-historical-data"
df_1 = "on.zip"
kg.api.dataset_download_files(dataset = data_1, path = df_1, unzip = True)
df_1 = pd.read_csv('on.zip/btcusd_1-min_data.csv')

## dataset2 Bitcoin/Eth - 2014-2025
data_2 = "kapturovalexander/bitcoin-and-ethereum-prices-from-start-to-2023"
df_2 = "on_1.zip
kg.api.dataset_download_files(dataset = data_2, path = df_2, unzip = True)
df_2 = pd.read_csv('{name}')

## dataset3 Bitcoin Fear/Greed
def get_fng_data():
    url = "https://api.alternative.me/fng/"
    params = {
        "limit": 0, format: "json", "date_format": "world"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()["data"]
    else:
        print("Error: ", response.status_code)
        return None

def convert_csv(data, filename="fear_greed_index.csv"):
    if not data:
        print("No data")
        return
    keys = data[0].keys()
    with open(filemame, mode="w", newline="",encoding = "utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)
    print(f"Data saved to {filename}")

df_3 = get_fng_data()
convert_csv(df_3)

## dataset4 Twitter scraping
query = "Bitcoin OR BTC -filter:retweets"
max_tweets = 1000000
tweets = []
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if i >= max_tweets:
        break
    sentiment = sia.polarity_scores(tweet.content)
    tweets.append([
        tweet.date, tweet.username, tweet.content, sentiment['compound'], sentiment['pos'], sentiment['neu'], sentiment['neg']
    ])

df_4 = pd.DataFrame(tweets, columns=['Date', 'Username', 'Tweet', 'Compound',  'Positive', 'Neutral', 'Negative'])
df_4.to_csv("bitcoin_sentiment.csv", index=False)

## dataset5 Reddit scraping
def scrape_reddit(subreddit_name, limit = 1000000)
    subreddit = reddit.subreddit(subreddit_name)
    posts = []

    for post in subreddit.hot(limit = limit):
        posts.append([
            post.title,
            post.score,
            post.created_utc,
            post.url
        ])
    df_5  = pd.DataFrame(posts,culumns=["Title","Score","Timestamp","URL"])
    df.to_csv("reddit_bitcoin.csv", index=False)
scrape_reddit("Bitcoin", limit=1000000) ## Feel free to add other subreddits

## dataset6 Google searches
def fetch_google(keyword="Bitcoin", timeframe="today 30-m"):
    pytrends=TrendReq(hl="en-US",tz=360)
    pytrends.build_payload([keyword],cat=0,timeframe=timeframe,geo="",gprop="")
    trends_data=pytrends.interest_over_time()
    if "isPartial" in trends_data.columns:
        trends_data.drop(columns=["isPartial"],inplace=True)
    trends_data.to_csv("google_trends_bitcoin.csv")
fetch_google("Bitcoin","today 30-m")